In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn import preprocessing

raw_csv_data= np.loadtxt('/content/drive/My Drive/Colab Notebooks/1.1 Audiobooks_data.csv.csv',delimiter = ',')

In [ ]:
unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all=raw_csv_data[:,-1]

Balance the datasheet

In [ ]:
num_one_target=int(np.sum(targets_all))
zero_target_counter=0
indices_to_remove=[]

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_target_counter+=1
    if zero_target_counter>num_one_target:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)

Standarise the input

In [ ]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

Shuffling the data

In [ ]:
shuffle_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffle_indices)

shuffle_inputs=scaled_inputs[shuffle_indices]
shuffel_targets=targets_equal_priors[shuffle_indices]

### Split the data into train,validation and test

In [ ]:
sample_count=shuffle_inputs.shape[0]
train_sample_count=int(0.8*sample_count)
validation_sample_count=int(0.1*sample_count)
test_sample_count=sample_count-train_sample_count-validation_sample_count
train_inputs=shuffle_inputs[:train_sample_count]
train_targets=shuffel_targets[:train_sample_count]

validation_inputs=shuffle_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_targets=shuffel_targets[train_sample_count:train_sample_count+validation_sample_count]

test_inputs=shuffle_inputs[train_sample_count+validation_sample_count:]
test_targets=shuffel_targets[train_sample_count+validation_sample_count:]

Save the file in .npz format

In [ ]:
np.savez('Audio_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audio_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audio_data_test',inputs=test_inputs,targets=train_targets)

Class for batching

In [ ]:
import numpy as np

#create a class that will do batching for the algorithm
# this code is extremely reusable. You should just change Audiobook_Data_Reader everywhere in the code
class Audiobook_Data_Reader():
  #dataset is a mandatory argument, while batch_sizeis optional
  #if you don't input batch_size it will automatically take it as None
  def __init__(self,dataset,batch_size=None):

    # The dataset that load in one of "train","validation","test".
    npz=np.load('Audio_data_{0}.npz'.format(dataset))

    #Two variable that take the value of inputs and the targets. Inputs are float and targets are integers
    self.inputs, self.targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

    # Counts the batch number, given the size you feed it later
    # If the batch size is none we are either validating or testing, so we want to take the data in a single batch
    if batch_size is None:
      self.batch_size = self.inputs.shape[0]
    else:
      self.batch_size= batch_size
    self.curr_batch=0
    self.batch_count=self.inputs.shape[0] // self.batch_size


    # A method whichh load the next batch
  def __next__(self):
    if self.curr_batch >= self.batch_count:
      self.curr_batch = 0
      raise StopIteration()

    # You slice the dataset in batch and the next function load them  one after the other
    batch_slice=slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
    inputs_batch = self.inputs[batch_slice]
    targets_batch = self.targets[batch_slice]
    self.curr_batch +=1 

    #one hot encoded the targets. In this example it's a bit superflows since we have a 0/1 column
    # classification task with more than one target column
    classes_num=2
    targets_one_hot=np.zeros((targets_batch.shape[0], classes_num))
    targets_one_hot[range(targets_batch.shape[0]),targets_batch]=1
    return inputs_batch, targets_one_hot

  def __iter__(self):
    return self

Machine Learning algorithm

In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 50

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32,[None,input_size])
targets = tf.placeholder(tf.int32,[None,output_size])

weight_1 = tf.get_variable("weight_1", [input_size,hidden_layer_size])
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])

output_1= tf.nn.relu(tf.matmul(inputs,weight_1) + biases_1)

weight_2 = tf.get_variable("weight_2", [hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])

output_2= tf.nn.relu(tf.matmul(output_1,weight_2) + biases_2)

weight_3 = tf.get_variable("weight_3", [hidden_layer_size,output_size])
biases_3 = tf.get_variable("biases_3",[output_size])

output=tf.matmul(output_2,weight_3) + biases_3

loss = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=targets)

mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate=0.01).minimize(mean_loss)

output_equal_targets=tf.equal(tf.arg_max(output,1),tf.arg_max(targets,1))

accuray = tf.reduce_mean(tf.cast(output_equal_targets,tf.float32))


sess = tf.InteractiveSession()
intializer = tf.global_variables_initializer()
sess.run(intializer)

batch_size=100

max_epochs=50

prev_validation_loss = 999999999

train_data=Audiobook_Data_Reader('train',batch_size)
validation_data=Audiobook_Data_Reader('validation',batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
for epoch_counter in range(max_epochs):
  curr_epoch_loss=0

  for inputs_data,target_data in train_data:
    _,batch_loss = sess.run([optimize,mean_loss],
                            feed_dict = {inputs: inputs_data, targets:target_data})
    curr_epoch_loss+=batch_loss

  curr_epoch_loss/=train_data.batch_count

  validation_loss=0.
  validation_accuracy=0.
  for inputs_data, target_data in validation_data:
    validation_loss,validation_accuracy = sess.run([mean_loss,accuray],
                                                   feed_dict = {inputs: inputs_data, targets:target_data})
    
  print('Epoch='+str(epoch_counter+1)+
        '.Training loss='+'{0:.3f}'.format(curr_epoch_loss)+
        '. Validation loss='+'{0:.3f}'.format(validation_loss)+
        '. Validation accuracy='+str(validation_accuracy*100)+
        '%')
  if validation_loss > prev_validation_loss:
    break
  prev_validation_loss=validation_loss

print('End of Training')

Epoch=1.Training loss=0.376. Validation loss=0.359. Validation accuracy=81.00000023841858%
Epoch=2.Training loss=0.356. Validation loss=0.338. Validation accuracy=81.99999928474426%
Epoch=3.Training loss=0.346. Validation loss=0.328. Validation accuracy=81.99999928474426%
Epoch=4.Training loss=0.344. Validation loss=0.319. Validation accuracy=81.99999928474426%
Epoch=5.Training loss=0.339. Validation loss=0.316. Validation accuracy=82.99999833106995%
Epoch=6.Training loss=0.338. Validation loss=0.312. Validation accuracy=81.00000023841858%
Epoch=7.Training loss=0.334. Validation loss=0.314. Validation accuracy=82.99999833106995%
End of Training


In [ ]:
test_data=Audiobook_Data_Reader('test')

for input_batch, target_batch in train_data:
  test_accuracy=sess.run([accuray],
                         feed_dict={inputs:input_batch, targets:target_batch})
test_accuracy_percentage=test_accuracy[0] * 100
print('Test accuracy:'+str(test_accuracy_percentage)+'%')

Test accuracy:75.99999904632568%


In [ ]:
print(test_accuracy)

[0.76]
